# Self-Driving Car Engineering
## Vehicle Detection and Tracking
A pipeline to detect and track cars in a video stream from an autonomous car's forward facing camera

## Step 1: Build and train a classifier to detect cars in an image